I became interested in understanding how network packets work when I read this book by Greg Conti:

In [1]:
from IPython.display import IFrame
IFrame('http://ecx.images-amazon.com/images/I/51gaXuc-WRL._SX258_BO1,204,203,200_.jpg', width=200, height=300)

It had some cool parallel plots to see what was being sent and received. The closest picture I could find is this:

In [2]:
IFrame('http://www.nsa.gatech.edu/honeynet/reports/images/vistool.gif', width=500, height=300)

In [3]:
IFrame('http://www.nsa.gatech.edu/honeynet/reports/images/secvis.jpg', width=500, height=300)

I was inspired to do a talk on Scapy by Julia Evans' talk at Pycon a few years ago, [Easy network hacking with scapy #MP47](https://www.youtube.com/watch?v=EnuF9ZR6MVc)
And her slides: [https://github.com/jvns/talks/blob/master/2014-06-mtlpy-scapy/easy-network-hacking-with-scapy.ipynb]()

# Packet manipulation without Scapy

I wanted to learn about networking, packets, protocols, etc. so I decided to first try and do things without a library that abstracts this away. I searched around and found the two libraries **impacket** and **pcapy**.

In [4]:
from IPython.display import IFrame
IFrame('https://pypi.python.org/pypi/impacket/0.9.13', width=1000, height=270)

In [5]:
IFrame('https://pypi.python.org/pypi/pcapy/0.10.8#what-is-pcapy', width=1000, height=170)

I had no idea what I was doing, so I googled for some already existing code snippets that I copied to understand how they worked:

In [6]:
IFrame('http://snipplr.com/view/3579/#viewsource', width=750, height=350)

In [7]:
IFrame('http://www.binarytides.com/code-a-packet-sniffer-in-python-with-pcapy-extension/#dd_start', width=1000, height=250)

First I use `help()` to find out about this packet

In [ ]:
import pcapy

help(pcapy)

We are going to use the `open_live` function. Since it requires a device as one of its arguments, let's look at what devices are available:

**/!\ Remember to run this ipython notebook in sudo mode to use pcapy and scapy**

In [ ]:
pcapy.findalldevs()

Let's capture on the wifi which is `'en1'`:

In [ ]:
# Arguments here are:
#   device
#   snaplen (maximum number of bytes to capture _per_packet_)
#   promiscious mode (1 for true)
#   timeout (in milliseconds)
device = 'en1'
snaplen = 65536
promiscuous = False
timeout = 100
pcapy.open_live(device, snaplen, promiscuous, timeout)

In [ ]:
packet_capture = pcapy.open_live(device, snaplen, promiscuous, timeout)

In [ ]:
help(packet_capture)
print '\n'.join(dir(packet_capture))
print
help(packet_capture.loop)

Let's use the `loop` function to get 20 packets.

In [ ]:
from impacket.ImpactDecoder import EthDecoder

## callback function for received packets
def receive_packets(header, packet):
    """header and packet make up the packet object"""
    print header
    print '-'*len(repr(header))
    data = EthDecoder().decode(packet)
    print data

    return

packet_limit = 20
packet_capture.loop(packet_limit, receive_packets)

In [ ]:
from struct import unpack
import socket
import datetime

print "Listening on %s: net=%s, mask=%s" % (
    device, packet_capture.getnet(), packet_capture.getmask()
)

def eth_addr(a):
    """Convert a string of 6 characters of ethernet address into a dash separated hex string"""
    b = "%.2x:%.2x:%.2x:%.2x:%.2x:%.2x" % (ord(a[0]) , ord(a[1]) , ord(a[2]), ord(a[3]), ord(a[4]) , ord(a[5]))
    return b

def parse_packet(packet, verbose=False):

    ## parse ethernet header
    eth_length = 14
    eth_header = packet[:eth_length]
    eth = unpack('!6s6sH', eth_header)
    eth_protocol = socket.ntohs(eth[2])

    d_mac = eth_addr(packet[0:6])
    s_mac = eth_addr(packet[6:12])

    if verbose:
        print
        print 'Ethernet header'
        print '---------------'
        print 'Destination MAC:     %s' % d_mac
        print 'Source MAC:          %s' % s_mac
        print 'Protocol:            %s' % eth_protocol

    ## parse IP packets, IP Protocol number = 8
    if eth_protocol == 8:

        ## parse IP header
        ## take first 20 characters for the ip header
        ip_header = packet[eth_length:20+eth_length]

        ## unpack them
        iph = unpack('!BBHHHBBH4s4s' , ip_header)

        version_ihl = iph[0]
        version = version_ihl >> 4
        ihl = version_ihl & 0xF

        iph_length = ihl * 4

        ttl = iph[5]
        protocol = iph[6]
        s_addr = socket.inet_ntoa(iph[8]);
        d_addr = socket.inet_ntoa(iph[9]);

        if verbose:
            print
            print 'IP header'
            print '---------'
            print 'Version:             %s' % version
            print 'IP Header Length:    %s' % ihl
            print 'TTL:                 %s' % ttl
            print 'Protocol:            %s' % protocol
            print 'Source Address:      %s' % s_addr
            print 'Destination Address: %s' % d_addr

        ## TCP protocol
        if protocol == 6 :
            protocol_acronym = 'TCP'

            t = iph_length + eth_length
            tcp_header = packet[t:t+20]

            ## unpack them
            tcph = unpack('!HHLLBBHHH' , tcp_header)

            source_port = tcph[0]
            dest_port = tcph[1]
            sequence = tcph[2]
            acknowledgement = tcph[3]
            doff_reserved = tcph[4]
            tcph_length = doff_reserved >> 4

            if verbose:
                print
                print 'TCP header'
                print '----------'
                print 'Source Port:         %s' % source_port
                print 'Dest Port:           %s' % dest_port
                print 'Sequence Number:     %s' % sequence
                print 'Acknowledgement:     %s' % acknowledgement
                print 'TCP header length:   %s' % tcph_length

            h_size = eth_length + iph_length + tcph_length * 4
            data_size = len(packet) - h_size

            ## get data from the packet
            data = packet[h_size:]

            if verbose:
                print 'Data: \n"""\n%s\n"""' % data

        ## ICMP Packets
        elif protocol == 1 :
            protocol_acronym = 'ICMP'

            u = iph_length + eth_length
            icmph_length = 4
            icmp_header = packet[u:u+4]

            #now unpack them :)
            icmph = unpack('!BBH' , icmp_header)

            icmp_type = icmph[0]
            code = icmph[1]
            checksum = icmph[2]

            if verbose:
                print
                print 'ICMP header'
                print '-----------'
                print 'Type :     %s' % icmp_type
                print 'Code :     %s' % code
                print 'Checksum : %s' % checksum

            h_size = eth_length + iph_length + icmph_length
            data_size = len(packet) - h_size

            #get data from the packet
            data = packet[h_size:]

            if verbose:
                print 'Data: %s' % data

        ## UDP packets
        elif protocol == 17 :
            protocol_acronym = 'UDP'

            u = iph_length + eth_length
            udph_length = 8
            udp_header = packet[u:u+8]

            #now unpack them :)
            udph = unpack('!HHHH' , udp_header)

            source_port = udph[0]
            dest_port = udph[1]
            length = udph[2]
            checksum = udph[3]

            if verbose:
                print
                print 'UDP header'
                print '----------'
                print 'Source Port: %s' % source_port
                print 'Dest Port:   %s' % dest_port
                print 'Length:      %s' % length
                print 'Checksum:    %s' % checksum

            h_size = eth_length + iph_length + udph_length
            data_size = len(packet) - h_size

            #get data from the packet
            data = packet[h_size:]

            if verbose:
                print 'Data: %s' % data

        #some other IP packet like IGMP
        else :
            print 'Protocol other than TCP/UDP/ICMP'
            protocol_acronym = 'Other'

        if 'source_port' not in locals():
            return s_addr, d_addr, protocol_acronym
        return s_addr, d_addr, source_port, dest_port, protocol_acronym


## start sniffing packets
try:
    while True:
        try:
            header, packet = packet_capture.next()
            addresses = parse_packet(packet)

            try:
                protocol = list(addresses).pop()
                addresses = zip(*(iter(addresses),) * 2)
            except TypeError:
                continue

            source_ip, dest_ip = addresses[0]
            try:
                source_port, dest_port = addresses[1]
            except IndexError:
                print protocol, source_ip, dest_ip
                continue

            dt = '{:%H:%M:%S}'.format(datetime.datetime.now())
            row = dt, protocol, source_ip, dest_ip, source_port, dest_port

             ## print to stdout whoich can then be piped to csv file
            print '\t'.join(map(str, row))

        except pcapy.PcapError, e:
            pass

except Exception, e:
    print e


# Using Scapy

**/!\ Remember to run this ipython notebook in sudo mode to use pcapy and scapy**

In [ ]:
from scapy.all import *

In [8]:
IFrame('http://www.secdev.org/projects/scapy/doc/usage.html#sniffing', width=1000, height=170)

In [ ]:
received_packets = sniff(filter="tcp", count=2)

In [ ]:
received_packets.nsummary()